In [ ]:
# automatically reloads imported files on edits
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

from pathlib import Path

from HH4b import utils
from HH4b.hh_vars import LUMI
import correctionlib
import mplhep as hep

hep.style.use(hep.style.CMS)

from constants import MASS_RANGE, PT_RANGE, PT_BINS, MASS_BINS

In [ ]:
YEARS = ["2022", "2022EE", "2023", "2023BPix"]
YEARS_COMBINED_DICT = {
    "2022All": ["2022", "2022EE"],
    "2023All": ["2023", "2023BPix"],
}
PROCESSED_PATH: Path = Path("processed/corr_DYLL_data.pkl")
(PROCESSED_PATH.parent).mkdir(parents=True, exist_ok=True)
REPROCESS: bool = True  # if True, reprocess from the skimmed ntuples

SAMPLES_DICT = {
    "data": [f"{key}_Run" for key in ["Muon"]],
    "ttbar": ["TTto4Q", "TTtoLNu2Q", "TTto2L2Nu"],
    "DYto2L": ["DYto2L"],
    "VV": ["WW", "WZ", "ZZ"],
}

PLOT_DIR = Path("plots/corr_DYLL_data")
PLOT_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_PATH = Path("corrs/DYLL_data.pkl")
(OUTPUT_PATH.parent).mkdir(parents=True, exist_ok=True)

P4 = ("Mass", "Pt", "Eta", "Phi")
columns = [("weight", 1)]
for i in range(2):
    for branch in P4:
        columns.append((f"Muon{i+1}{branch}", 1))

In [ ]:
def get_POG_MUO_path(year: str, filename: str = "muon_Z.json.gz") -> Path:
    corr_dir = Path("POG_MUO")
    if year == "2022":
        subdir = "2022_Summer22"
    elif year == "2022EE":
        subdir = "2022_Summer22EE"
    elif year == "2023":
        subdir = "2023_Summer23"
    elif year == "2023BPix":
        subdir = "2023_Summer23BPix"
    else:
        raise ValueError(f"Unknown year: {year}")
    return corr_dir / subdir / filename


CORR_DICT = {}

for year in YEARS:
    corr_path = get_POG_MUO_path(year)
    if not corr_path.exists():
        raise FileNotFoundError(f"Correction file for {year} not found at {corr_path}")

    # Load the correction file
    corr = correctionlib.CorrectionSet.from_file(str(corr_path))
    CORR_DICT[year] = corr

In [ ]:
def get_dimuon_PtEtaPhiMass(df: pd.DataFrame) -> pd.Series:
    """Calculate the dimuon mass from the muon 4-vectors."""
    muon1_pt = df[("Muon1Pt", 0)]
    muon1_eta = df[("Muon1Eta", 0)]
    muon1_phi = df[("Muon1Phi", 0)]
    muon1_mass = df[("Muon1Mass", 0)]
    muon1_px = muon1_pt * np.cos(muon1_phi)
    muon1_py = muon1_pt * np.sin(muon1_phi)
    muon1_pz = muon1_pt * np.sinh(muon1_eta)
    muon1_E = np.sqrt(muon1_px**2 + muon1_py**2 + muon1_pz**2 + muon1_mass**2)

    muon2_pt = df[("Muon2Pt", 0)]
    muon2_eta = df[("Muon2Eta", 0)]
    muon2_phi = df[("Muon2Phi", 0)]
    muon2_mass = df[("Muon2Mass", 0)]
    muon2_px = muon2_pt * np.cos(muon2_phi)
    muon2_py = muon2_pt * np.sin(muon2_phi)
    muon2_pz = muon2_pt * np.sinh(muon2_eta)
    muon2_E = np.sqrt(muon2_px**2 + muon2_py**2 + muon2_pz**2 + muon2_mass**2)

    dimuon_px = muon1_px + muon2_px
    dimuon_py = muon1_py + muon2_py
    dimuon_pz = muon1_pz + muon2_pz
    dimuon_E = muon1_E + muon2_E

    dimuon_pt = np.sqrt(dimuon_px**2 + dimuon_py**2)
    dimuon_eta = np.arcsinh(dimuon_pz / np.sqrt(dimuon_px**2 + dimuon_py**2))
    dimuon_phi = np.arctan2(dimuon_py, dimuon_px)
    dimuon_mass = np.sqrt(dimuon_E**2 - (dimuon_px**2 + dimuon_py**2 + dimuon_pz**2))

    return {
        "DimuonPt": dimuon_pt,
        "DimuonEta": dimuon_eta,
        "DimuonPhi": dimuon_phi,
        "DimuonMass": dimuon_mass,
    }


def get_POG_MUO_corr(df: pd.DataFrame, corr: correctionlib.CorrectionSet) -> pd.DataFrame:
    corr = CORR_DICT[year]

    eta1 = df[("Muon1Eta", 0)]
    pt1 = df[("Muon1Pt", 0)]
    eta2 = df[("Muon2Eta", 0)]
    pt2 = df[("Muon2Pt", 0)]

    sf_nominal = 1.0
    rel_errs_up_sq = 0.0
    rel_errs_down_sq = 0.0

    # HLT correction
    HLT_key = (
        "NUM_Mu50_or_CascadeMu100_or_HighPtTkMu100_DEN_CutBasedIdGlobalhighPtId_and_TkIsoLoose"
    )
    hlt_nom = corr[HLT_key].evaluate(eta1, pt1, "nominal")
    hlt_up = corr[HLT_key].evaluate(eta1, pt1, "systup")
    hlt_down = corr[HLT_key].evaluate(eta1, pt1, "systdown")

    sf_nominal *= hlt_nom
    rel_errs_up_sq += ((hlt_up - hlt_nom) / hlt_nom) ** 2
    rel_errs_down_sq += ((hlt_down - hlt_nom) / hlt_nom) ** 2

    for pt, eta in zip([pt1, pt2], [eta1, eta2]):
        # Loose ID
        looseId_key = "NUM_LooseID_DEN_TrackerMuons"
        loose_nom = corr[looseId_key].evaluate(eta, pt, "nominal")
        loose_up = corr[looseId_key].evaluate(eta, pt, "systup")
        loose_down = corr[looseId_key].evaluate(eta, pt, "systdown")

        sf_nominal *= loose_nom
        rel_errs_up_sq += ((loose_up - loose_nom) / loose_nom) ** 2
        rel_errs_down_sq += ((loose_down - loose_nom) / loose_nom) ** 2

        # High Pt ID
        highPtId_key = "NUM_TrkHighPtID_DEN_TrackerMuons"
        highPtId_nom = corr[highPtId_key].evaluate(eta, pt, "nominal")
        highPtId_up = corr[highPtId_key].evaluate(eta, pt, "systup")
        highPtId_down = corr[highPtId_key].evaluate(eta, pt, "systdown")

        sf_nominal *= highPtId_nom
        rel_errs_up_sq += ((highPtId_up - highPtId_nom) / highPtId_nom) ** 2
        rel_errs_down_sq += ((highPtId_down - highPtId_nom) / highPtId_nom) ** 2

    # Calculate asymmetric uncertainties
    total_relative_error_up = np.sqrt(rel_errs_up_sq)
    total_relative_error_down = np.sqrt(rel_errs_down_sq)

    sf_up = sf_nominal * (1 + total_relative_error_up)
    sf_down = sf_nominal * (1 - total_relative_error_down)

    weight = df["finalWeight"]
    return {
        "finalWeight": weight * sf_nominal,
        "finalWeight_MUO_up": weight * sf_up,
        "finalWeight_MUO_down": weight * sf_down,
    }

In [ ]:
if REPROCESS or not PROCESSED_PATH.exists():
    path_dir = (
        "/ceph/cms/store/user/zichun/bbbb/skimmer/DYLLtoDataHT25June4_v12_ZbbSFZMuMu_zbb-DYLL-data/"
    )

    events_dict = {}
    # Process eras
    for year in YEARS:
        dataframes = {
            **utils.load_samples(
                data_dir=path_dir,
                samples=SAMPLES_DICT,
                year=year,
                columns=utils.format_columns(columns),
                variations=True,
                weight_shifts=[],
            )
        }
        for sample, df in dataframes.items():
            # Add Dimuon kinematics
            df = df.assign(**get_dimuon_PtEtaPhiMass(df))
            # Apply POG MUO corrections
            df = df.assign(**get_POG_MUO_corr(df, corr=CORR_DICT[year]))
            dataframes[sample] = df

        events_dict[year] = dataframes

    # Combine into years
    events_combined = {year: {} for year in YEARS_COMBINED_DICT.keys()}
    for sample in SAMPLES_DICT:
        for combined_year, year_list in YEARS_COMBINED_DICT.items():
            events_combined[combined_year][sample] = pd.concat(
                [events_dict[year][sample] for year in year_list if sample in events_dict[year]]
            )

    # Save the processed data
    with PROCESSED_PATH.open("wb") as f:
        pd.to_pickle(events_combined, f)
else:
    # Load the processed data
    with PROCESSED_PATH.open("rb") as f:
        events_combined = pd.read_pickle(f)

In [ ]:
cuts = {
    "DimuonPt": PT_RANGE,
    "DimuonMass": MASS_RANGE,
}

events_combined_sel = {}
for year, events in events_combined.items():
    events_combined_sel[year] = {}
    for sample, df in events.items():
        # Apply cuts
        for cut_name, (low, high) in cuts.items():
            df = df[(df[cut_name] >= low) & (df[cut_name] <= high)]
        # Reset index after filtering
        df = df.reset_index(drop=True)

        # Store the selected events
        events_combined_sel[year][sample] = df

In [ ]:
for feature_name, feature_label, bins in zip(
    ["DimuonMass", "DimuonPt"],
    [r"$M^{\mu\mu}$ [GeV]", r"$p_\mathrm{T}^{\mu\mu}$ [GeV]"],
    [MASS_BINS, PT_BINS],
):
    for year in events_combined_sel:
        fig, (ax1, ax2) = plt.subplots(
            2, 1, figsize=(12, 10), gridspec_kw={"height_ratios": [3, 1], "hspace": 0.1}
        )
        bin_centers = (bins[:-1] + bins[1:]) / 2

        # Initialize arrays for MC sum and error calculation
        mc_total = np.zeros(len(bins) - 1)
        mc_error_sq = np.zeros(len(bins) - 1)  # Sum of squares for statistical error
        mc_syst_error_sq = np.zeros(len(bins) - 1)  # Sum of squares for systematic error
        data_hist = None
        data_error = None

        # Plot individual samples and calculate MC sum
        for sample in events_combined_sel[year]:
            feature = events_combined_sel[year][sample][feature_name].to_numpy()
            weight = events_combined_sel[year][sample]["finalWeight"].to_numpy()

            # Create histogram
            hist, _ = np.histogram(feature, bins=bins, weights=weight)

            # Plot on main axis
            ax1.hist(
                feature,
                bins=bins,
                weights=weight,
                label=sample,
                histtype="step",
            )

            if sample == "data":
                data_hist = hist
                # For data, error is sqrt(N) where N is unweighted counts
                unweighted_hist, _ = np.histogram(feature, bins=bins)
                data_error = np.sqrt(unweighted_hist)
            else:
                # Sum MC samples
                mc_total += hist
                # Statistical error: sum of weight squares
                weight_sq_hist, _ = np.histogram(feature, bins=bins, weights=weight**2)
                mc_error_sq += weight_sq_hist

                # Systematic error from MUO variations
                weight_up = events_combined_sel[year][sample]["finalWeight_MUO_up"].to_numpy()
                weight_down = events_combined_sel[year][sample]["finalWeight_MUO_down"].to_numpy()

                hist_up, _ = np.histogram(feature, bins=bins, weights=weight_up)
                hist_down, _ = np.histogram(feature, bins=bins, weights=weight_down)

                # Systematic uncertainty: max deviation from nominal
                syst_var = np.maximum(np.abs(hist_up - hist), np.abs(hist_down - hist))
                mc_syst_error_sq += syst_var**2

        # Total MC error: quadrature sum of statistical and systematic
        mc_stat_error = np.sqrt(mc_error_sq)
        mc_syst_error = np.sqrt(mc_syst_error_sq)
        mc_total_error = np.sqrt(mc_error_sq + mc_syst_error_sq)

        # Main plot formatting
        ax1.set_ylabel("Events")
        ax1.set_yscale("log")
        ax1.legend()
        ax1.tick_params(axis="x", labelbottom=False)  # Hide x-axis labels on top plot

        # Set x-axis limits - extend to 2000 for pT plots
        if feature_name == "DimuonPt":
            ax1.set_xlim(200, 2000)

        # Ratio plot
        if data_hist is not None:
            # Calculate ratio and its error
            ratio = np.divide(
                data_hist, mc_total, out=np.zeros_like(data_hist), where=mc_total != 0
            )

            # Error propagation for ratio: sqrt((σ_data/MC)² + (data*σ_MC_total/MC²)²)
            ratio_error = np.zeros_like(ratio)
            mask = mc_total > 0
            ratio_error[mask] = np.sqrt(
                (data_error[mask] / mc_total[mask]) ** 2
                + (data_hist[mask] * mc_total_error[mask] / mc_total[mask] ** 2) ** 2
            )

            # Plot ratio with error bars
            ax2.errorbar(bin_centers, ratio, yerr=ratio_error, fmt="ko", markersize=3, capsize=2)

            # Add horizontal line at y=1
            ax2.axhline(y=1, color="red", linestyle="--", alpha=0.7)

            # Ratio plot formatting
            ax2.set_xlabel(feature_label)
            ax2.set_ylabel("Data / MC")
            ax2.set_ylim(0.5, 1.5)  # Adjust as needed
            ax2.grid(True, alpha=0.3)

            # Set x-axis limits for ratio plot too
            if feature_name == "DimuonPt":
                ax2.set_xlim(PT_BINS[0], PT_BINS[-1])

        plt.tight_layout()

        hep.cms.label(
            ax=ax1,
            label="Work in Progress",
            data=True,
            year=year.replace("All", ""),
            com=13.6,
            lumi=(round(LUMI[year] / 1000, 2)),
        )

        plt.savefig(PLOT_DIR / f"{feature_name}_{year}.pdf", bbox_inches="tight")
        plt.show()

# Derive $f_\mathrm{DYLL \to Data} (p_\mathrm{T}^{\mu\mu})$

In [ ]:
signal_sample = "DYto2L"

SF_dict = {}

for year in events_combined_sel:
    data_counts = np.zeros(len(PT_BINS) - 1)
    signal_counts = np.zeros(len(PT_BINS) - 1)
    background_counts = np.zeros(len(PT_BINS) - 1)

    signal_counts_up = np.zeros(len(PT_BINS) - 1)
    signal_counts_down = np.zeros(len(PT_BINS) - 1)
    background_counts_up = np.zeros(len(PT_BINS) - 1)
    background_counts_down = np.zeros(len(PT_BINS) - 1)

    # Statistical error tracking
    data_sumw2 = np.zeros(len(PT_BINS) - 1)
    signal_sumw2 = np.zeros(len(PT_BINS) - 1)
    background_sumw2 = np.zeros(len(PT_BINS) - 1)

    # Process each sample
    for sample in events_combined_sel[year]:
        pt = events_combined_sel[year][sample]["DimuonPt"].to_numpy()
        weight = events_combined_sel[year][sample]["finalWeight"].to_numpy()

        # Create histogram for this sample in PT bins
        hist, _ = np.histogram(pt, bins=PT_BINS, weights=weight)
        hist_sumw2, _ = np.histogram(pt, bins=PT_BINS, weights=weight**2)

        if sample == "data":
            data_counts = hist
            data_sumw2 = hist_sumw2
        elif sample == signal_sample:
            signal_counts = hist
            signal_sumw2 = hist_sumw2
            # Signal systematic variations
            weight_up = events_combined_sel[year][sample]["finalWeight_MUO_up"].to_numpy()
            weight_down = events_combined_sel[year][sample]["finalWeight_MUO_down"].to_numpy()
            signal_counts_up, _ = np.histogram(pt, bins=PT_BINS, weights=weight_up)
            signal_counts_down, _ = np.histogram(pt, bins=PT_BINS, weights=weight_down)
        else:
            # Background: other MCs
            background_counts += hist
            background_sumw2 += hist_sumw2
            # Get systematic variations for background
            weight_up = events_combined_sel[year][sample]["finalWeight_MUO_up"].to_numpy()
            weight_down = events_combined_sel[year][sample]["finalWeight_MUO_down"].to_numpy()
            hist_up, _ = np.histogram(pt, bins=PT_BINS, weights=weight_up)
            hist_down, _ = np.histogram(pt, bins=PT_BINS, weights=weight_down)
            background_counts_up += hist_up
            background_counts_down += hist_down

    # Calculate nominal scale factors and systematic variations
    numerator = data_counts - background_counts
    numerator_up = data_counts - background_counts_up
    numerator_down = data_counts - background_counts_down

    scale_factor = np.divide(
        numerator, signal_counts, out=np.zeros_like(numerator), where=signal_counts > 0
    )
    scale_factor_syst_up = np.divide(
        numerator_up, signal_counts_up, out=np.zeros_like(numerator_up), where=signal_counts_up > 0
    )
    scale_factor_syst_down = np.divide(
        numerator_down,
        signal_counts_down,
        out=np.zeros_like(numerator_down),
        where=signal_counts_down > 0,
    )

    # Calculate statistical errors
    data_stat_err = np.sqrt(data_sumw2)
    background_stat_err = np.sqrt(background_sumw2)
    signal_stat_err = np.sqrt(signal_sumw2)

    # Statistical error propagation for scale factor
    numerator_stat_err = np.sqrt(data_stat_err**2 + background_stat_err**2)
    numerator_rel_err = np.divide(
        numerator_stat_err,
        np.abs(numerator),
        out=np.zeros_like(numerator_stat_err),
        where=np.abs(numerator) > 0,
    )
    signal_rel_err = np.divide(
        signal_stat_err, signal_counts, out=np.zeros_like(signal_stat_err), where=signal_counts > 0
    )
    sf_rel_stat_err = np.sqrt(numerator_rel_err**2 + signal_rel_err**2)
    sf_stat_err = np.abs(scale_factor) * sf_rel_stat_err

    # Systematic errors (difference from nominal)
    sf_syst_err_up = np.abs(scale_factor_syst_up - scale_factor)
    sf_syst_err_down = np.abs(scale_factor_syst_down - scale_factor)

    # Combine systematic and statistical errors in quadrature
    sf_total_err_up = np.sqrt(sf_syst_err_up**2 + sf_stat_err**2)
    sf_total_err_down = np.sqrt(sf_syst_err_down**2 + sf_stat_err**2)

    # Final up/down variations
    scale_factor_up = scale_factor + sf_total_err_up
    scale_factor_down = scale_factor - sf_total_err_down

    SF_dict[year] = {
        "nominal": scale_factor,
        "up": scale_factor_up,
        "down": scale_factor_down,
        "pt": PT_BINS,
    }

# Save to pickle file
with OUTPUT_PATH.open("wb") as f:
    pd.to_pickle(SF_dict, f)

print(f"Scale factors saved to {OUTPUT_PATH}")

In [ ]:
# Plot the scale factors
for year in SF_dict:
    sf = SF_dict[year]

    fig, ax = plt.subplots(figsize=(10, 6))

    # Plot nominal scale factor
    ax.errorbar(
        (PT_BINS[:-1] + PT_BINS[1:]) / 2,
        sf["nominal"],
        xerr=(PT_BINS[1:] - PT_BINS[:-1]) / 2,
        yerr=sf["up"] - sf["nominal"],
        fmt="o",
        color="blue",
        markersize=5,
        capsize=3,
    )

    # plot 1
    ax.axhline(y=1, color="red", linestyle="--", label="Nominal SF = 1")

    ax.set_xlabel(r"$p_\mathrm{T}^{\mu\mu}$ [GeV]")
    ax.set_ylabel("Scale Factor")
    # ax.set_xlim(PT_BINS[0], PT_BINS[-1])
    ax.set_ylim(0.5, 1.5)
    # ax.grid(True)

    hep.cms.label(
        ax=ax,
        label="Work in Progress",
        data=True,
        year=year.replace("All", ""),
        com=13.6,
        lumi=(round(LUMI[year] / 1000, 2)),
    )

    plt.tight_layout()
    plt.savefig(PLOT_DIR / f"SF_{year}.pdf", bbox_inches="tight")
    plt.close()